# Base de données répertoriant les taux de délinquance/criminalité (1996-2022 en France)

(NB : Les liens hypertextes présent dans ce markdown et les suivant renvoient aux sites internet sur lesquelles les tableaux de données ont été téléchargées et ne correspondent pas aux URL de téléchargement, l'exécution des cellules de code permet de télécharger automatiquement les jeu de données)

Nous avons récupéré, grâce à l'open data du ministère de l'intérieur ([ici](https://www.data.gouv.fr/fr/datasets/chiffres-departementaux-mensuels-relatifs-aux-crimes-et-delits-enregistres-par-les-services-de-police-et-de-gendarmerie-depuis-janvier-1996/)) les données relatives aux crimes et délit enregistrés dans chaque département et en France entre janvier 1996 et août 2022. Ce jeu de données a été produit successivement par le Ministère de l'Intérieur puis par le SSMSI depuis sa création en 2014. 

Issu directement des applications informatiques de gestion, il n'est pas retraité et reste assez brut. Entre autre, il ne nous donne accès qu'au nombre d'infractions (classées par type) par département, nous l'avons donc complété avec des données d'estimation de population pour obtenir des taux d'infractions plutôt que des chiffres bruts. Nous avons ensuite ajoutée à notre base de données d'indices de localisation pour nous permettre de réaliser des représentations géographiques dans notre partie de statistiques descriptives.

L'exécution des codes prenant parfois plusieurs minutes, les bases de données intermédiaires et finales produites ici sont sauvegardées dans le dossier *data_police* au format excel ou csv.

### Récupération et nettoyage des données du Ministère de l'Intérieur relative aux crimes et délits


In [2]:
import requests
import pandas as pd
import os
import zipfile

In [3]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
# URL de téléchargement du fichier
url = "https://static.data.gouv.fr/resources/chiffres-departementaux-mensuels-relatifs-aux-crimes-et-delits-enregistres-par-les-services-de-police-et-de-gendarmerie-depuis-janvier-1996/20221031-102847/tableaux-4001-ts.xlsx" 

# Dossier de destination pour enregistrer le fichier téléchargé
dossier_destination = "data_police"
os.makedirs(dossier_destination, exist_ok=True)

# Chemin du fichier téléchargé
fichier_telecharge = os.path.join(dossier_destination, "fichier.xlsx")

# Téléchargement du fichier
print("Téléchargement en cours...")
response = requests.get(url, stream=True)
with open(fichier_telecharge, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)
print("Téléchargement terminé:", fichier_telecharge)

# Chargement des onglets du fichier Excel en DataFrames
print("Chargement des onglets dans des DataFrames...")
data = pd.read_excel(fichier_telecharge, sheet_name=None)  # `sheet_name=None` charge tous les onglets

# Fusion des onglets dans un unique DataFrame avec une colonne pour identifier le département (ou la zone)
données_police = pd.concat(
    [df.assign(Zone=nom_onglet) for nom_onglet, df in data.items()],
    ignore_index=True
)

# Affichage du DataFrame combiné
print("\nDataFrame combiné avec tous les onglets :")
print(données_police)

# Sauvegarde du DataFrame combiné dans un fichier Excel
excel_output_path = os.path.join(dossier_destination, "données police.xlsx")
données_police.to_excel(excel_output_path, index=False)

print(f"Le DataFrame combiné a été sauvegardé dans : {excel_output_path}")

# Suppression du fichier téléchargé 
os.remove(fichier_telecharge)
print("Fichier téléchargé supprimé.")

Téléchargement en cours...
Téléchargement terminé: data_police/fichier.xlsx
Chargement des onglets dans des DataFrames...

DataFrame combiné avec tous les onglets :
       Index                                      libellé index  _2022_08  \
0          1             Règlements de compte entre malfaiteurs        14   
1          2       Homicides pour voler et à l'occasion de vols         2   
2          3                     Homicides pour d'autres motifs        88   
3          4  Tentatives d'homicides pour voler et à l'occas...        15   
4          5          Tentatives homicides pour d'autres motifs       315   
...      ...                                                ...       ...   
11658    103  Infractions à l'exercice d'une profession règl...         3   
11659    104  Infractions au droit de l'urbanisme et de la c...         2   
11660    105                                   Fraudes fiscales         0   
11661    106            Autres délits économiques et financiers  

On sépare notre premier jeu de données (données police) en deux pour obtenir une base de données départementale et une base nationale.


In [5]:
#Données pour la france entière
df_france_court = données_police[données_police['Zone'].isin(['France_Entière', 'France_Métro'])]

#Affichage
print("DataFrame la France entière:")
print(df_france_court)

#Sauvegarde un fichier Excel
excel_output_path = os.path.join(dossier_destination, "données france court.xlsx")
df_france_court.to_excel(excel_output_path, index=False)

print(f"Le DataFrame combiné a été sauvegardé dans : {excel_output_path}")

DataFrame la France entière:
     Index                                      libellé index  _2022_08  \
0        1             Règlements de compte entre malfaiteurs        14   
1        2       Homicides pour voler et à l'occasion de vols         2   
2        3                     Homicides pour d'autres motifs        88   
3        4  Tentatives d'homicides pour voler et à l'occas...        15   
4        5          Tentatives homicides pour d'autres motifs       315   
..     ...                                                ...       ...   
209    103  Infractions à l'exercice d'une profession règl...       130   
210    104  Infractions au droit de l'urbanisme et de la c...       328   
211    105                                   Fraudes fiscales        62   
212    106            Autres délits économiques et financiers      1168   
213    107                                      Autres délits     13941   

     _2022_07  _2022_06  _2022_05  _2022_04  _2022_03  _2022_02  _2022

In [6]:
# Sélectionner les lignes où 'Onglet' n'est ni 'France_Entière' ni 'France_Métro'
df_dep_court = données_police[~données_police['Zone'].isin(['France_Entière', 'France_Métro'])]

# Affichage du dataframe obtenu
print("DataFrame sans France entière ni France Métro :")
print(df_dep_court)

# Sauvegarde dans un fichier Excel
excel_output_path = os.path.join(dossier_destination, "données départements court.xlsx")
df_dep_court.to_excel(excel_output_path, index=False)

print(f"Le DataFrame combiné a été sauvegardé dans : {excel_output_path}")


DataFrame sans France entière ni France Métro :
       Index                                      libellé index  _2022_08  \
214        1             Règlements de compte entre malfaiteurs         0   
215        2       Homicides pour voler et à l'occasion de vols         0   
216        3                     Homicides pour d'autres motifs         0   
217        4  Tentatives d'homicides pour voler et à l'occas...         0   
218        5          Tentatives homicides pour d'autres motifs         0   
...      ...                                                ...       ...   
11658    103  Infractions à l'exercice d'une profession règl...         3   
11659    104  Infractions au droit de l'urbanisme et de la c...         2   
11660    105                                   Fraudes fiscales         0   
11661    106            Autres délits économiques et financiers         6   
11662    107                                      Autres délits        66   

       _2022_07  _2022_06  

Le format des tableaux de données actuel indique les dates sur une ligne (voir ci-dessous). 

| Index  | Libellé | Zone | _2022_08 | _2022_07 | ... | _1996_01 |
|-----------|----------|----------|----------|----------|----------|----------|
| Type 1    | Libellé  1 | France | Valeur 1 | Valeur 3 | Valeur 5 | Valeur 8 |
| Type 2    | Libellé  2 | France | Valeur 2 | Valeur 4 | Valeur 6 | Valeur 9 |

Nous préférions avoir une colonne indiquant les dates car il nous semblait plus simple de filtrer par date et d'ajouter de nouvelle colonnes d'information ainsi (exemple ci-dessous).

| Index | Libellé | Zone | Date |
|-------|---------|------|------|
| Type 1 | Libellé 1 | Zone a | _2022_08 |
| Type 2 | Libellé 2 | Zone a | _2022_08 |
| ... | ... | ... | ... |
| Type 1 | Libellé 1 | Zone f | _2012_05 |
| ... | ... | ... | ... |
| Type 34 | Libellé 34 | Zone z | _1996_01 |

Nous avons donc restructurer les deux tableaux de données (df_france_court et df_dep_court) en format long.


In [7]:
# Transformer le tableau (échelle nationale) en format "long"
df_france = df_france_court.melt(id_vars=['Index', 'libellé index', 'Zone'],  # Colonnes fixes
                  var_name='Date',  # Nom pour la colonne de dates
                  value_name='Nombre')  # Nom pour la colonne des valeurs

# Affichage
print(df_france)

# Sauvegarde dans un fichier Excel
excel_output_path = os.path.join(dossier_destination, "données france.xlsx")
df_france.to_excel(excel_output_path, index=False)

print(f"Le DataFrame a été sauvegardé dans : {excel_output_path}")

       Index                                      libellé index  \
0          1             Règlements de compte entre malfaiteurs   
1          2       Homicides pour voler et à l'occasion de vols   
2          3                     Homicides pour d'autres motifs   
3          4  Tentatives d'homicides pour voler et à l'occas...   
4          5          Tentatives homicides pour d'autres motifs   
...      ...                                                ...   
68475    103  Infractions à l'exercice d'une profession règl...   
68476    104  Infractions au droit de l'urbanisme et de la c...   
68477    105                                   Fraudes fiscales   
68478    106            Autres délits économiques et financiers   
68479    107                                      Autres délits   

                 Zone      Date  Nombre  
0      France_Entière  _2022_08    14.0  
1      France_Entière  _2022_08     2.0  
2      France_Entière  _2022_08    88.0  
3      France_Entière  _202

In [8]:
# Transformer le tableau (échelle départementale) en format "long"
df_dep = df_dep_court.melt(
    id_vars=['Index', 'libellé index', 'Zone'],  # Colonnes fixes
    var_name='Date',  # Nom pour la colonne de dates
    value_name='Nombre'  # Nom pour la colonne des valeurs
)

# Affichage
print(df_dep)

# Sauvegarde dans un fichier CSV
csv_output_path = os.path.join(dossier_destination, "données départements.csv")
df_dep.to_csv(csv_output_path, index=False)

#Le dataframe obtenu a un assez grand nombre de ligne (autour de 3 millions) et il n'était pas possible de le push sur github sans le zipper

# Chemin du fichier ZIP
zip_output_path = os.path.join(dossier_destination, "données départements.zip")

# Création d'un fichier ZIP et ajout du fichier CSV
with zipfile.ZipFile(zip_output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_output_path, arcname="données départements.csv")

# Suppression du fichier CSV non zippé
os.remove(csv_output_path)

print(f"Le fichier zippé a été sauvegardé dans : {zip_output_path}")


         Index                                      libellé index Zone  \
0            1             Règlements de compte entre malfaiteurs   01   
1            2       Homicides pour voler et à l'occasion de vols   01   
2            3                     Homicides pour d'autres motifs   01   
3            4  Tentatives d'homicides pour voler et à l'occas...   01   
4            5          Tentatives homicides pour d'autres motifs   01   
...        ...                                                ...  ...   
3663675    103  Infractions à l'exercice d'une profession règl...  988   
3663676    104  Infractions au droit de l'urbanisme et de la c...  988   
3663677    105                                   Fraudes fiscales  988   
3663678    106            Autres délits économiques et financiers  988   
3663679    107                                      Autres délits  988   

             Date  Nombre  
0        _2022_08     0.0  
1        _2022_08     0.0  
2        _2022_08     0.0  

In [9]:
# Vérification de l'existence de doublons dans le tableau départemental
dep_doublons = df_dep[df_dep.duplicated(keep=False)]
print(dep_doublons)

# Vérification de l'existence de doublons dans le tableau national
france_doublons = df_france[df_france.duplicated(keep=False)]
print(france_doublons)

Empty DataFrame
Columns: [Index, libellé index, Zone, Date, Nombre]
Index: []
Empty DataFrame
Columns: [Index, libellé index, Zone, Date, Nombre]
Index: []


Après vérification, aucun doublons n'est présent dans les données récupérées du Ministère de l'Intérieur (il était indiqué sur data.gouv.fr que c'était une éventualité).

En revanche, les données concernants les territoires d'outre mer ne sont pas disponibles avant 2000, certaines valeurs seront donc manquante dans notre jeu de données.

### Ajout des données relatives à la population

Pour obtenir les taux d'infractions, nous devons ajouter à nos deux tableaux de données, les estimations de population récupérée sur le site de l'INSEE à l'échelle nationale ([ici](https://www.insee.fr/fr/statistiques/serie/000436387#Tableau)) et départementale ([et là](https://catalogue-donnees.insee.fr/fr/explorateur/DS_ESTIMATION_POPULATION)). 

Contrairement aux données nationales, les données départementales récupérées sont disponibles en open data avec une granularité annuelle et non pas mensuelle ce qui introduit une incertitude dans les futurs calculs de taux d'infractions.


In [10]:
# URL de téléchargement du fichier
url = "https://www.insee.fr/fr/statistiques/serie/telecharger/csv/000436387?ordre=antechronologique&transposition=donneescolonne&periodeDebut=1&anneeDebut=1996&periodeFin=8&anneeFin=2022&revision=sansrevisions" 

# Dossier de destination
destination = "/home/onyxia/work/Python-pour-la-data-science-2A/data/data_pop"
os.makedirs(destination, exist_ok=True)

# Chemin du fichier zip téléchargé
fichier_zip = os.path.join(destination, "fichier.zip")

# Téléchargement du fichier zip
print("Téléchargement en cours...")
response = requests.get(url, stream=True)
with open(fichier_zip, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)
print("Téléchargement terminé:", fichier_zip)

# Décompression du fichier zip
print("Décompression en cours...")
with zipfile.ZipFile(fichier_zip, 'r') as zip_ref:
    zip_ref.extractall(destination)
print("Décompression terminée.")

# Vérification des fichiers extraits
fichiers = os.listdir(destination)
print("Fichiers présents après décompression :", fichiers)

# Localisation et chargement du fichier valeurs_mensuelles.csv
fichier_cible = os.path.join(destination, "valeurs_mensuelles.csv")
if not os.path.exists(fichier_cible):
    raise FileNotFoundError("Le fichier valeurs_mensuelles.csv n'a pas été trouvé.")

print("Chargement du fichier valeurs_mensuelles.csv en DataFrame...")
data_valeurs_mensuelles = pd.read_csv(fichier_cible, delimiter=';', encoding='utf-8') 

# Affichage des premières lignes du DataFrame
print("\nAperçu du DataFrame :")
print(data_valeurs_mensuelles)

# Suppression des fichiers inutiles
print("Nettoyage des fichiers inutiles...")
for fichier in fichiers:
    chemin_fichier = os.path.join(destination, fichier)
    if chemin_fichier != fichier_cible:  # On conserve seulement valeurs_mensuelles.csv
        os.remove(chemin_fichier)
        print(f"Fichier supprimé : {chemin_fichier}")

print("Processus terminé. Seul le fichier valeurs_mensuelles.csv a été conservé.")

Téléchargement en cours...
Téléchargement terminé: /home/onyxia/work/Python-pour-la-data-science-2A/data/data_pop/fichier.zip
Décompression en cours...
Décompression terminée.
Fichiers présents après décompression : ['fichier.zip', 'caractéristiques.csv', 'valeurs_mensuelles.csv']
Chargement du fichier valeurs_mensuelles.csv en DataFrame...

Aperçu du DataFrame :
                  Libellé  \
0                  idBank   
1    Dernière mise à jour   
2                 Période   
3                 2022-08   
4                 2022-07   
..                    ...   
318               1996-05   
319               1996-04   
320               1996-03   
321               1996-02   
322               1996-01   

    Démographie - Population au début du mois - France métropolitaine Codes  
0                                            000436387                  NaN  
1                                     29/11/2024 12:00                  NaN  
2                                                  

In [11]:
print(data_valeurs_mensuelles)
print("Noms des colonnes :", data_valeurs_mensuelles.columns)


                  Libellé  \
0                  idBank   
1    Dernière mise à jour   
2                 Période   
3                 2022-08   
4                 2022-07   
..                    ...   
318               1996-05   
319               1996-04   
320               1996-03   
321               1996-02   
322               1996-01   

    Démographie - Population au début du mois - France métropolitaine Codes  
0                                            000436387                  NaN  
1                                     29/11/2024 12:00                  NaN  
2                                                  NaN                  NaN  
3                                                65843                    P  
4                                                65820                    P  
..                                                 ...                  ...  
318                                              57976                    A  
319                        

In [12]:
data_valeurs_mensuelles.drop(columns=['Codes'], inplace=True)

In [13]:
print(data_valeurs_mensuelles)

                  Libellé  \
0                  idBank   
1    Dernière mise à jour   
2                 Période   
3                 2022-08   
4                 2022-07   
..                    ...   
318               1996-05   
319               1996-04   
320               1996-03   
321               1996-02   
322               1996-01   

    Démographie - Population au début du mois - France métropolitaine  
0                                            000436387                 
1                                     29/11/2024 12:00                 
2                                                  NaN                 
3                                                65843                 
4                                                65820                 
..                                                 ...                 
318                                              57976                 
319                                              57963                 
320

In [14]:
# Suppression des trois premières lignes
data_valeurs_mensuelles = data_valeurs_mensuelles.iloc[3:]

# Réinitialiser les index (facultatif)
data_valeurs_mensuelles.reset_index(drop=True, inplace=True)

print(data_valeurs_mensuelles)

     Libellé Démographie - Population au début du mois - France métropolitaine
0    2022-08                                              65843               
1    2022-07                                              65820               
2    2022-06                                              65794               
3    2022-05                                              65770               
4    2022-04                                              65756               
..       ...                                                ...               
315  1996-05                                              57976               
316  1996-04                                              57963               
317  1996-03                                              57951               
318  1996-02                                              57943               
319  1996-01                                              57936               

[320 rows x 2 columns]


In [15]:
# Transformer le format de la colonne "date"
data_valeurs_mensuelles["Libellé"] = data_valeurs_mensuelles["Libellé"].str.replace("-", "_").apply(lambda x: f"_{x}")

print("\nAprès transformation :")
print(data_valeurs_mensuelles)


Après transformation :
      Libellé  \
0    _2022_08   
1    _2022_07   
2    _2022_06   
3    _2022_05   
4    _2022_04   
..        ...   
315  _1996_05   
316  _1996_04   
317  _1996_03   
318  _1996_02   
319  _1996_01   

    Démographie - Population au début du mois - France métropolitaine  
0                                                65843                 
1                                                65820                 
2                                                65794                 
3                                                65770                 
4                                                65756                 
..                                                 ...                 
315                                              57976                 
316                                              57963                 
317                                              57951                 
318                                              57

On joint les deux DataFrames sur les colonnes de dates. On note que notre DateFrame comportant les données de la police comprend les données pour la France entière (territoire d'Outre-Mer inclus) et pour la France métropolitaine. En revanche les estimations de population mensuelle de l'INSEE ne prennent en compte que la France métropolitaine. On supprimera donc dans un prochain temps les lignes de notre tableau correspondant à la France entière faute de trouver les chiffres adéquat pour la population. L'inner join effectué ci-dessous a associé les estimations de populations de la France métropolitaine à la fois à la France métropolitaine et à la France entière, la première partie du tableau comporte donc des informations fausses, on les ignorera pour le moment car elles seront supprimées.

In [25]:
# Inner join sur des colonnes avec des noms différents
resultat = pd.merge(
    df_france,
    data_valeurs_mensuelles,
    left_on='Date',  # Colonnes de df_france
    right_on='Libellé', # Colonnes de data_valeurs_mensuelles
    how='inner'  # Type de jointure
)

#Les colonnes 'Date' et 'Libellé' sont identiques donc on supprime 'Libellé' (nom moins évocateurs)
resultat = resultat.drop(columns=["Libellé"])

# Renommer la colonne 'Démographie - Population au début du mois - France métropolitaine' car c'est bien trop long
resultat = resultat.rename(columns={'Démographie - Population au début du mois - France métropolitaine': 'Population'})

#Affichage du résultat
print(resultat)

       Index                                      libellé index  \
0          1             Règlements de compte entre malfaiteurs   
1          2       Homicides pour voler et à l'occasion de vols   
2          3                     Homicides pour d'autres motifs   
3          4  Tentatives d'homicides pour voler et à l'occas...   
4          5          Tentatives homicides pour d'autres motifs   
...      ...                                                ...   
68475    103  Infractions à l'exercice d'une profession règl...   
68476    104  Infractions au droit de l'urbanisme et de la c...   
68477    105                                   Fraudes fiscales   
68478    106            Autres délits économiques et financiers   
68479    107                                      Autres délits   

                 Zone      Date  Nombre Population  
0      France_Entière  _2022_08    14.0      65843  
1      France_Entière  _2022_08     2.0      65843  
2      France_Entière  _2022_08    8

URL de téléchargement du fichier zipé répertoriant la population française totale par mois depuis aout 1996
https://www.insee.fr/fr/statistiques/serie/telecharger/csv/000436387?ordre=antechronologique&transposition=donneescolonne&periodeDebut=1&anneeDebut=1996&periodeFin=8&anneeFin=2022&revision=sansrevisions

URL de déléchargement du fichier répertoriant la population par département depuis 1996 (granularité annuelle pas mensuelle mais on fait comme on peut)
https://api.insee.fr/melodi/data/DS_ESTIMATION_POPULATION/to-csv?maxResult=10000&SEX=_T&AGE=_T&TIME_PERIOD=1996&TIME_PERIOD=1997&TIME_PERIOD=1998&TIME_PERIOD=1999&TIME_PERIOD=2000&TIME_PERIOD=2001&TIME_PERIOD=2002&TIME_PERIOD=2003&TIME_PERIOD=2004&TIME_PERIOD=2005&TIME_PERIOD=2006&TIME_PERIOD=2007&TIME_PERIOD=2008&TIME_PERIOD=2009&TIME_PERIOD=2010&TIME_PERIOD=2011&TIME_PERIOD=2012&TIME_PERIOD=2013&TIME_PERIOD=2014&TIME_PERIOD=2015&TIME_PERIOD=2016&TIME_PERIOD=2017&TIME_PERIOD=2018&TIME_PERIOD=2019&TIME_PERIOD=2020&TIME_PERIOD=2021&TIME_PERIOD=2022&GEO=DEP&optionCsv=%7B%22format%22:%22csv%22,%22affichage%22:%22codes%22,%22decimales%22:%22point%22,%22periodeColonne%22:false,%22lang%22:%22fr%22%7D


### Calcul des taux de délinquance

### Ajout des indicateurs géographiques